In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(1)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
import matplotlib.pyplot as plt
import PIL.Image as Image
import math
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
import json
from tensorflow.keras.losses import mse
import os
import glob
import random
import sys
ros_path = '/opt/ros/kinetic/lib/python2.7/dist-packages'

if ros_path in sys.path:

    sys.path.remove(ros_path)

import cv2

sys.path.append('/opt/ros/kinetic/lib/python2.7/dist-packages')

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
import datetime
from tensorflow.keras.layers import Dense, Input, concatenate, Conv2D, MaxPooling2D, Flatten, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
%matplotlib inline

# Cityscapes Data

In [4]:
from camera import *

left_imgs = []

right_imgs = []

disparity = []

camera_x = []
camera_baseline = []
camera_y = []
camera_z = []
camera_pitch = []
camera_fx = []
camera_fy = []
camera_u0 = []
camera_v0 = []

x_world = []
y_world = []
z_world = []

test_left_img_paths = []
test_disp_img_paths = []
test_cam_img_paths = []

cam = CIsoCamera()

data_path = "/home/talha/Documents/Cityscapes Dataset/"
test_disparity_img_path = data_path+"disparity/test/cityscapesDataset/"
test_leftImg8bit_img_path = data_path+"leftImg8bit/test/cityscapesDataset/"
test_camera_path = data_path+"camera/test/cityscapesDataset/"

for i in os.listdir(test_disparity_img_path):
   
    test_disp_img_paths.append(test_disparity_img_path+i)
    
    test_left_img_paths.append(test_leftImg8bit_img_path+i.split(".")[0][:-9]+"leftImg8bit.png")
    
    test_cam_img_paths.append(test_camera_path+i.split(".")[0][:-9]+"camera.json")
    


for i in range(len(test_disp_img_paths)):

    
        
    dispImg = Image.open(test_disp_img_paths[i])
    dispNp = dispImg.resize((112,112), Image.ANTIALIAS)
    dispNp = np.array(dispNp,dtype=np.float)
    dispNp = ( dispNp - 1. ) / 256.
    disp = dispNp[5,5] + 1
    
    disparity.append(disp)
        
    leftImg = Image.open(test_left_img_paths[i])
    leftImg = leftImg.resize((112,112), Image.ANTIALIAS)
    left_imgs.append(np.array(leftImg))
    dispImg = Image.open(test_disp_img_paths[i])
    dispNp = dispImg.resize((112,112), Image.ANTIALIAS)
    dspimg = Image.new("RGB", dispNp.size)
    dspimg.paste(dispNp)
    rightImg = np.array(leftImg) - np.array(dspimg)
    right_imgs.append(np.array(rightImg))
        
    cam.loadFromJson(test_cam_img_paths[i])
        
    camera_x.append(cam.extrinsic.x)
    camera_baseline.append(cam.extrinsic.baseline)
    camera_y.append(cam.extrinsic.y)
    camera_z.append(cam.extrinsic.z)
    camera_pitch.append(cam.extrinsic.pitch)
    camera_fx.append(cam.intrinsic.fx)
    camera_fy.append(cam.intrinsic.fy)
    camera_u0.append(cam.intrinsic.u0)
    camera_v0.append(cam.intrinsic.v0)
        
        
    u = 5
    v = 5
    
        
    
       
    xCam = (cam.intrinsic.fx * cam.extrinsic.baseline) / disp
    yCam = - (xCam / cam.intrinsic.fx) * (u - cam.intrinsic.u0)
    zCam = (xCam / cam.intrinsic.fy) * (cam.intrinsic.v0 - v)

    yWorld = yCam + cam.extrinsic.y
    xWorld = xCam * math.cos(cam.extrinsic.pitch) + zCam*math.sin(cam.extrinsic.pitch) + cam.extrinsic.x
    zWorld = - xCam * math.sin(cam.extrinsic.pitch) + zCam*math.cos(cam.extrinsic.pitch) + cam.extrinsic.z
        
    x_world.append(xWorld)
    y_world.append(yWorld)
    z_world.append(zWorld)
        
        

        

In [5]:
print ("Training Dataset: ",len(left_imgs)*0.7, "Test Dataset: ", len(left_imgs)*0.3)

Training Dataset:  1067.5 Test Dataset:  457.5


In [6]:
c = list(zip(left_imgs, right_imgs, camera_fx, camera_fy, camera_u0, camera_v0, camera_baseline, disparity, camera_x, camera_y, camera_z, camera_pitch, x_world, y_world, z_world))
random.shuffle(c)
left_imgs, right_imgs, camera_fx, camera_fy, camera_u0, camera_v0, camera_baseline, disparity, camera_x, camera_y, camera_z, camera_pitch, x_world, y_world, z_world = zip(*c)
left_imgs, right_imgs, camera_fx, camera_fy, camera_u0, camera_v0, camera_baseline, disparity, camera_x, camera_y, camera_z, camera_pitch, x_world, y_world, z_world = list(left_imgs), list(right_imgs), list(camera_fx), list(camera_fy), list(camera_u0), list(camera_v0), list(camera_baseline), list(disparity), list(camera_x), list(camera_y), list(camera_z), list(camera_pitch), list(x_world), list(y_world), list(z_world)



In [7]:
np.save("../Data/lir_cs.npy",left_imgs)
np.save("../Data/rir_cs.npy",right_imgs)
np.save("../Data/fxr_cs.npy",camera_fx)
np.save("../Data/fyr_cs.npy",camera_fy) 
np.save("../Data/u0r_cs.npy",camera_u0) 
np.save("../Data/v0r_cs.npy",camera_v0) 
np.save("../Data/br_cs.npy",camera_baseline)
np.save("../Data/dr_cs.npy",disparity) 
np.save("../Data/txr_cs.npy",camera_x) 
np.save("../Data/tyr_cs.npy",camera_y) 
np.save("../Data/tzr_cs.npy",camera_z) 
np.save("../Data/pr_cs.npy",camera_pitch)
np.save("../Data/xr_cs.npy",x_world)
np.save("../Data/yr_cs.npy",y_world) 
np.save("../Data/zr_cs.npy",z_world)